# Part 6 - Project: Build Your Own AI Agent

Build a complete Research Assistant with tools and memory.

In [ ]:
!pip install -q langgraph langchain langchain-openai

In [ ]:
import os
import getpass

if not os.getenv('OPENAI_API_KEY'):
    os.environ['OPENAI_API_KEY'] = getpass.getpass('Enter OpenAI API key: ')

## Step 1: Define Tools

In [ ]:
from langchain.tools import tool
import json
from datetime import datetime

@tool
def web_search(query: str) -> str:
    '''Search the web.'''
    return f'Search results for: {query}'

@tool
def save_notes(content: str, title: str) -> str:
    '''Save research notes.'''
    note = {'title': title, 'content': content, 'timestamp': datetime.now().isoformat()}
    try:
        with open('notes.json', 'r') as f:
            notes = json.load(f)
    except FileNotFoundError:
        notes = []
    notes.append(note)
    with open('notes.json', 'w') as f:
        json.dump(notes, f)
    return f'Saved: {title}'

@tool
def read_notes(query: str = '') -> str:
    '''Read saved notes.'''
    try:
        with open('notes.json', 'r') as f:
            notes = json.load(f)
        return str(notes)
    except FileNotFoundError:
        return 'No notes found'

tools = [web_search, save_notes, read_notes]

## Step 2: Create Agent

In [ ]:
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver

model = ChatOpenAI(model='gpt-4')
system_message = '''You are a Research Assistant. You can:
- Search the web
- Save notes
- Recall past research'''

memory = MemorySaver()
agent = create_react_agent(model, tools, state_modifier=system_message, checkpointer=memory)

## Step 3: Use the Agent

In [ ]:
config = {'configurable': {'thread_id': 'user_1'}}

# Research something
response = agent.invoke(
    {'messages': [('user', 'Research artificial intelligence and save key findings')]},
    config
)
print(response['messages'][-1].content)

In [ ]:
# Recall notes
response = agent.invoke(
    {'messages': [('user', 'What did you save about AI?')]},
    config
)
print(response['messages'][-1].content)